## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-19-13-02-00 +0000,wsj,Larry Summers Resigns From OpenAI Board Amid E...,https://www.wsj.com/tech/ai/larry-summers-resi...
1,2025-11-19-13-00-53 +0000,wapo,Cow cuddling gives farms boost amid dropping d...,https://www.washingtonpost.com/lifestyle/2025/...
2,2025-11-19-13-00-38 +0000,nypost,NYPD Commissioner Jessica Tisch accepts Mayor-...,https://nypost.com/2025/11/19/us-news/nypd-com...
3,2025-11-19-13-00-05 +0000,nyt,Russia Pounds Ukraine Even as Zelensky Aims to...,https://www.nytimes.com/2025/11/19/world/europ...
4,2025-11-19-13-00-00 +0000,nypost,LI mom’s impossible decision saves ‘lifeless’ ...,https://nypost.com/2025/11/19/us-news/li-moms-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
125,trump,54
6,epstein,27
175,saudi,18
105,new,18
74,house,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
205,2025-11-18-22-58-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...,149
374,2025-11-18-13-24-46 +0000,cbc,Saudi crown prince MBS unaware of Khashoggi ki...,https://www.cbc.ca/news/world/washington-saudi...,142
224,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...,139
159,2025-11-19-01-33-54 +0000,nypost,Trump announces Saudi Arabia will be designate...,https://nypost.com/2025/11/18/us-news/trump-an...,127
268,2025-11-18-19-44-18 +0000,nypost,Congress votes to send Epstein files release b...,https://nypost.com/2025/11/18/us-news/house-vo...,120


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
205,149,2025-11-18-22-58-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
374,88,2025-11-18-13-24-46 +0000,cbc,Saudi crown prince MBS unaware of Khashoggi ki...,https://www.cbc.ca/news/world/washington-saudi...
100,61,2025-11-19-08-42-26 +0000,nypost,Family of high school California tennis star k...,https://nypost.com/2025/11/19/us-news/parents-...
224,58,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...
235,44,2025-11-18-21-46-47 +0000,latimes,"'Played with fire, got burned': GOP control of...",https://www.latimes.com/politics/story/2025-11...
296,42,2025-11-18-18-04-02 +0000,nyt,Zelensky Will Try to Revive Peace Talks With R...,https://www.nytimes.com/2025/11/18/world/europ...
286,37,2025-11-18-18-52-13 +0000,nypost,Google’s Sundar Pichai warns of AI spending ‘i...,https://nypost.com/2025/11/18/business/google-...
310,34,2025-11-18-17-18-09 +0000,nypost,NY Gov. Kathy Hochul enlists Wall Street execs...,https://nypost.com/2025/11/18/business/ny-gov-...
328,33,2025-11-18-16-20-13 +0000,cbc,North Carolina becomes latest battleground as ...,https://www.cbc.ca/news/world/us-north-carolin...
61,33,2025-11-19-10-50-41 +0000,nyt,Trump Administration Gives Three Mile Island N...,https://www.nytimes.com/2025/11/19/us/nuclear-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
